# Banco de Dados

1. **Modelar o Banco de Dados**

In [ ]:
-- Active: 1706665252780@@127.0.0.1@5432@postgres
psql -U postgres

CREATE DATABASE e_commerce;

\c e_commerce;

In [ ]:
-- Active: 1706665252780@@127.0.0.1@5432@e_commerce@public
CREATE TABLE products (
    id_product SERIAL PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    description TEXT,
    category VARCHAR(255), 
    currentPrice INTEGER,
    mainImg VARCHAR(255),
    thumbnail VARCHAR(255)
);


In [ ]:
-- Active: 1706665252780@@127.0.0.1@5432@e_commerce@public
CREATE TABLE users (
    id_user SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) NOT NULL,
    password VARCHAR NOT NULL
);

In [ ]:
-- Active: 1706665252780@@127.0.0.1@5432@e_commerce@public
CREATE TABLE sales (
    id_sale SERIAL PRIMARY KEY,
    user_id INTEGER REFERENCES users(id_user),
    product_id INTEGER REFERENCES products(id_product),
    purchase_date DATE NOT NULL,
    purchase_price INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    status VARCHAR(20) NOT NULL CHECK (status IN ('Finalizado', 'Esperando Pagamento'))
);

2. **Criar o DER**

<img src="e-commerce.png">

3. **Criar o script SQL**

In [ ]:
INSERT INTO products (id_product, title, description, currentPrice, mainImg, thumbnail)
VALUES (1, 'Camisa Karasuno nº10', 'Camisa do time de Volei Karasuno', 60, 'https://i.ibb.co/18VWf4k/image-product-1.jpg', 'https://i.ibb.co/C7F6rCT/image-product-1-thumbnail.jpg');

In [ ]:
DROP TABLE sales;
DROP TABLE products;

# API

4. **Construir a rota de pegar todos os produtos**

In [ ]:
const postgre = require('../database')
const productController = {
    getAll: async(req, res) => {
        try {
            const { rows } = await postgre.query("select * from products")
            res.json({msg: "OK", data: rows})
        } catch (error) {
            res.json({msg: error.msg})
        }
    }
}

In [ ]:
router.get("/", productController.getAll)

5. **Construir a rota de filtrar os produtos**

In [ ]:
getById: async(req, res) => {
    try {
        const { rows } = await postgre.query("select * from products where id_product = $1", [req.params.id])

        if (rows[0]) {
            return res.json({msg: "OK", data: rows})
        }

        res.status(404).json({msg: "not found"})
    } catch (error) {
        res.json({msg: error.msg})
    }
}

In [ ]:
router.get("/:id", productController.getById)

6. **Construir a rota de cadastrar um usuário**

In [ ]:
create: async (req, res) => {
    try {
        const { name, email, password } = req.body;

        const sql = 'INSERT INTO users(name, email, password) VALUES($1, $2, $3) RETURNING *';

        const { rows } = await postgre.query(sql, [name, email, password]);

        res.json({ msg: "OK", data: rows[0] });

    } catch (error) {
        res.json({ msg: error.msg });
    }
}

In [ ]:
router.post("/create", userController.create);

7. **Construir a rota de login**

In [ ]:
authenticateUser: async (req, res) => {
    try {
        const { email, password } = req.body;

        const query = 'SELECT * FROM users WHERE email = $1 AND password = $2';
        const { rows } = await postgre.query(query, [email, password]);

        console.log("Rows:", rows); 

        if (rows.length > 0) {
            const user = rows[0];
            console.log("User:", user); 

            const token = jwt.sign({ email: user.email, id: user.id_user, name: user.name }, secret, { expiresIn: '1h' });

            return res.status(200).json({ token, name: user.name });
        } else {
            return res.status(401).json({ msg: "Unauthorized" });
        }
    } catch (error) {
        console.error(error);
        res.status(500).json({ msg: "Internal Server Error" });
    }
}

In [ ]:
router.post("/authenticate", userController.authenticateUser);

8. **Construir a rota de pegar todas as vendas e seus produtos de um usuário específico**

9. **Construir a rota de cadastrar uma venda**